In [1]:
import sqlite3 as db

import pandas as pd
from alpaca.data.historical import CryptoHistoricalDataClient
# from alpaca.data.bars import
from alpaca.data.requests import CryptoBarsRequest
from alpaca.data.timeframe import TimeFrame
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import GetAssetsRequest
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
from alpaca.data.live.crypto import CryptoDataStream
from utils.clients import AlpacaClient
from utils.mappings import alpaca_stream_message_map,alpaca_stream_col_map
import sys
import json
import datetime as dt
import Keys
import asyncio
import threading
import asyncio
from pandas.tseries.offsets import Hour


In [2]:
t_frame=TimeFrame(1,unit='h')


In [3]:
symbol="ETHUSDT"
pair="ETH/USD"
base_asset=pair.split('/')[0]
quote_asset=pair.split('/')[1]
history_path='Trade_history/trade.db'

conn=db.connect(history_path)
history_df=pd.read_sql(f'select * from {symbol}_trade_history',conn)
history_df.head(5)

,idx,step,date,position_index,position,real_position,portfolio_valuation,portfolio_distribution_asset,portfolio_distribution_fiat,portfolio_distribution_borrowed_asset,...,feature_log_volume_20,feature_log_return_25,feature_log_volume_25,feature_log_return_30,feature_log_volume_30,ds,symbol,unique_id,dynamic_feature__0,dynamic_feature__1
0,-1,0,2024-06-03 10:52:14.993620,0,0.0,0.0,11897.083188,0.7874,8922.498586,0,...,0.974330,0.429621,0.655696,0.432442,0.202964,2024-06-03 17:59:59.999000,ETHUSDT,8,0,0
1,-1,0,2024-06-04 15:32:41.674590,0,0.0,0.0,11920.741196,0.7829,8930.650371,0,...,-0.033560,0.566785,-0.345679,0.395415,-0.200718,2024-06-04 22:59:59.999000,ETHUSDT,8,0,0
2,-1,0,2024-06-04 15:32:42.825506,0,0.0,0.0,11920.741196,0.7829,8930.650371,0,...,-0.029978,0.566806,-0.343017,0.395071,-0.197461,2024-06-04 22:59:59.999000,ETHUSDT,8,0,0
3,-1,1,2024-06-04 15:32:43.874930,0,-0.5,-0.5,11920.258949,0.7803,8940.402491,0,...,-0.029978,0.566806,-0.343017,0.395071,-0.197461,2024-06-04 22:59:59.999000,ETHUSDT,8,0,0
4,-1,0,2024-06-04 15:33:51.614968,0,0.0,0.0,11922.498410,0.7803,8940.402491,0,...,-0.010170,0.566898,-0.328377,0.393574,-0.179491,2024-06-04 22:59:59.999000,ETHUSDT,8,0,0


In [4]:
api_key=Keys.alpaca_api_key
api_secret=Keys.alpaca_api_secret

In [5]:
paper_url='wss://stream.data.paper.alpaca.markets/v1beta3/crypto/us'
live_url='wss://stream.data.alpaca.markets/v1beta3/crypto/us'


In [6]:
sock=CryptoDataStream(api_key=api_key,secret_key=api_secret,raw_data=True)

sock._is_running=True

In [7]:
example_bars=[{
  "T": "b",
  "S": "BTC/USD",
  "o": 71856.1435,
  "h": 71856.1435,
  "l": 71856.1435,
  "c": 71856.1435,
  "v": 0,
  "t": "2024-03-12T10:37:00Z",
  "n": 0,
  "vw": 0
},{
  "T": "b",
  "S": "BTC/USD",
  "o": 71856.1435,
  "h": 71856.1435,
  "l": 71856.1435,
  "c": 71856.1435,
  "v": 0,
  "t": "2024-03-12T10:37:00Z",
  "n": 0,
  "vw": 0
}]

In [8]:
from pandas.tseries.offsets import DateOffset,Hour

In [9]:
bar_time=pd.Timestamp("2024-03-12T11:59:00Z")
bar_time.strftime("%H:%M")

'11:59'

In [10]:
bar_time=pd.Timestamp("2024-03-12T11:00:00Z")
print(bar_time.strftime("%H:%M"),"####### is on hour?",bar_time.minute==0)

11:00 ####### is on hour? True


In [11]:
data=pd.DataFrame(example_bars)

data=data.rename(columns=alpaca_stream_col_map)
data['date_close']=data['date_close'].apply(pd.Timestamp)
data['bar_type']=data['bar_type'].map(alpaca_stream_message_map)

data

,bar_type,symbol,open,high,low,close,volume,date_close,n,vw
0,Bar,BTC/USD,71856.1435,71856.1435,71856.1435,71856.1435,0,2024-03-12 10:37:00+00:00,0,0
1,Bar,BTC/USD,71856.1435,71856.1435,71856.1435,71856.1435,0,2024-03-12 10:37:00+00:00,0,0


In [12]:
import nest_asyncio
nest_asyncio.apply()

In [13]:
symbol='ETHUSD'

In [14]:
self = AlpacaClient(api_key, api_secret,symbol=pair,time_frame='1h',paper=True)


In [15]:
self.account()

{'id': UUID('dec2a319-e7fb-4e20-80b5-f87228b2bf52'),
 'account_number': 'PA3IWKESH30I',
 'status': <AccountStatus.ACTIVE: 'ACTIVE'>,
 'crypto_status': <AccountStatus.ACTIVE: 'ACTIVE'>,
 'currency': 'USD',
 'buying_power': '50173.12',
 'regt_buying_power': '50173.12',
 'daytrading_buying_power': '0',
 'non_marginable_buying_power': '25086.56',
 'cash': '25086.56',
 'accrued_fees': '0',
 'pending_transfer_out': None,
 'pending_transfer_in': '0',
 'portfolio_value': '95206.72',
 'pattern_day_trader': False,
 'trading_blocked': False,
 'transfers_blocked': False,
 'account_blocked': False,
 'created_at': datetime.datetime(2024, 6, 5, 14, 13, 42, 888418, tzinfo=datetime.timezone.utc),
 'trade_suspended_by_user': False,
 'multiplier': '2',
 'shorting_enabled': True,
 'equity': '95206.72',
 'last_equity': '98713.8339808224',
 'long_market_value': '70120.16',
 'short_market_value': '0',
 'initial_margin': '0',
 'maintenance_margin': '0',
 'last_maintenance_margin': '0',
 'sma': '99995.07',
 'd

In [16]:
self.symbol

'ETH/USD'

In [17]:
start_date=(dt.datetime.now()-pd.Timedelta(hours=48*2))
data=self.get_historical_data(start_date=start_date)
data

,symbol,date_close,open,high,low,close,volume,trade_count,vwap
0,ETH/USD,2024-06-03 16:00:00+00:00,3795.8600,3795.8600,3762.129000,3768.252,1.172982,2.0,3784.721313
1,ETH/USD,2024-06-03 17:00:00+00:00,3766.6425,3781.3200,3758.000000,3779.850,0.770078,1.0,3773.370000
2,ETH/USD,2024-06-03 18:00:00+00:00,3782.4250,3785.7850,3775.614500,3778.950,0.574934,5.0,3781.054553
3,ETH/USD,2024-06-03 19:00:00+00:00,3776.6875,3779.4360,3767.190000,3771.670,8.348204,206.0,3771.216379
4,ETH/USD,2024-06-03 20:00:00+00:00,3775.3000,3782.3865,3771.613000,3771.613,0.102469,5.0,3775.888703
...,...,...,...,...,...,...,...,...,...
98,ETH/USD,2024-06-07 18:00:00+00:00,3721.4095,3721.4095,3598.014888,3676.825,57.610046,39.0,3652.978828
99,ETH/USD,2024-06-07 19:00:00+00:00,3674.4865,3702.2485,3659.267000,3688.380,1.167807,15.0,3680.177141
100,ETH/USD,2024-06-07 20:00:00+00:00,3680.0060,3692.0700,3670.400000,3689.905,0.191169,3.0,3674.537783
101,ETH/USD,2024-06-07 21:00:00+00:00,3687.5315,3697.5655,3683.572000,3694.100,0.000000,0.0,0.000000


In [18]:
price=self.ticker_price(self.symbol.replace('/',''))
price

APIError: {"message":"invalid symbol: ETHUSD does not match ^[A-Z]+/[A-Z]+$"}


In [ ]:
trade_info=self.trade_client.get_asset(self.symbol)
trade_info=trade_info.dict()
trade_info

In [ ]:
trade_info['min_trade_increment']*2,(trade_info['min_trade_increment'])
trade_info['price_increment']

In [ ]:
trade_info=self.get_trade_rules()
min_asset_size=trade_info['min_asset_size']
min_quote_size=min_asset_size*price
prec=trade_info['quote_asset_precision']

size=min_asset_size*price
quant=round(size,prec)
quant_norm=max([min_asset_size,quant])
quant,quant_norm,min_asset_size

In [ ]:
quant=quant_norm
params = {
    'symbol': self.symbol,
    'side':"buy",
    'type': 'MARKET',
    # 'qty':min_asset_size
    'notional':quant,
    "time_in_force":'ioc'
    }

In [ ]:
# preparing market order
market_order_data = self.order(params=params)
market_order_data

In [ ]:
positions = self.trade_client.get_all_positions()
position_list=[p.dict() for p in positions]
pos_frame=pd.DataFrame.from_dict(position_list)
pos_frame['symbol']
pos_frame=pos_frame.set_index('symbol')
pos_frame['qty_available'].to_dict()

In [ ]:
def get_asset_info(self,pair):
    info=trading_client.get_asset(pair)
    info=info.dict()


In [ ]:
def set_trade_rules(self):
    assets=trading_client.get_all_assets()
    asset_df=pd.DataFrame.from_dict([asset.dict() for asset in assets])
    asset_df=asset_df.set_index('symbol')
    asset_df.loc[pair]

In [ ]:
account=trading_client.get_account()
account.dict()

In [ ]:

cash=account['cash']
int(cash)

In [ ]:
renderer=LiveRenderer('Trade_history/trade.db')

In [ ]:
renderer=LiveRenderer('Trade_history/trade.db')
renderer.run()


In [ ]:
sql_query = """SELECT name FROM sqlite_master  
  WHERE type='table';"""

cursor.execute(sql_query)
render_names=[t[0] for t in cursor.fetchall()]
name=render_names[0]
df=pd.read_sql(f'select * from {name}',conn)
df

In [ ]:
# df.columns=[c.replace('data_','') for c in df.columns]
# df['date']=df['date'].apply(pd.to_datetime)
# df=df.set_index('date')
df

In [ ]:
# df=df[~df.close.isna()]
mask=df[(df['date']=='62')==True].index
df.loc[mask,'date']=df.loc[mask,'ds'].copy()


In [ ]:
df

In [ ]:
df.to_sql(render_names[0],conn,if_exists='replace',index=False)

In [ ]:
import datetime

In [ ]:
df.iloc[[-1]].to_dict('records')[0]

In [ ]:
pd.Timestamp(datetime.datetime.now())